<a href="https://colab.research.google.com/github/donghuna/PromptGenerate/blob/main/trainingPlan-1B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import locale
def getpreferredencoding(do_not_reload=True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!pip install peft
!pip install datasets
!pip install bitsandbytes
!pip install --upgrade torch
!pip install transformers
!pip install huggingface_hub
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datetime import datetime
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, PeftModel, get_peft_model
from datasets import load_dataset
import pandas as pd

In [3]:
from google.colab import userdata

from huggingface_hub import login
login(token=userdata.get('HUG_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
## 4bit quant ###

model_path = "meta-llama/Llama-3.2-1B-Instruct"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Load Tokenizer and Model
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    model_max_length=512,
    padding_side="left",
    add_eos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    # device_map="auto",
    # trust_remote_code=True,
    # use_auth_token=True,
)

model.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-0

In [6]:
# ### Load Model Normaly ###

# model_path = "meta-llama/Llama-3.1-8B-Instruct"

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Load Tokenizer
# tokenizer = AutoTokenizer.from_pretrained(
#     model_path,
#     model_max_length=512,
#     padding_side="left",
#     add_eos_token=True,
# )
# tokenizer.pad_token = tokenizer.eos_token

# # Load Model without quantization
# model = AutoModelForCausalLM.from_pretrained(
#     model_path,
#     torch_dtype=torch.float16,  # GPU 메모리 절약을 위해 float16 사용 (필요시 float32로 변경 가능)
#     device_map="auto",  # 모델을 자동으로 GPU로 로드
# )

# # model.eval()
# model.train()


In [7]:
# dataset = load_dataset("koutch/staqc", 'sca_python')
dataset = load_dataset("donghuna/StaQC-plan")
dataset

README.md:   0%|          | 0.00/470 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9873 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question_id', 'question', 'snippet', 'similarQuestion', 'confidence', 'plan'],
        num_rows: 9873
    })
})

In [8]:
def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=512,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

In [9]:
def generate_and_tokenize_prompt(data_point):
    full_prompt = f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Given a original competitive programming problem, generate a concrete planning to solve the problem.
The provided retrieved question and code are examples to help generate the plan.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
### Original Problem to be solved :
{data_point["similarQuestion"]}

### Retrieved Problem (similar question) :
{data_point["question"]}

### Retrieved code snippet (related to the retrieved question) :
{data_point["snippet"]}

### Similarity score (between original problem and retrieved question) :
{data_point["confidence"]}
--------------------
Important: You should give only the planning to solve the problem. Do not add extra explanation or words.
Let's think step by step.

### Planning to solve the problem in 6 lines or less:<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
{data_point["plan"]}
<|eot_id|><|end_of_text|>
"""
    return tokenize(full_prompt)

In [10]:
import numpy as np

# 유사도 점수로부터 가중치를 계산
def calculate_weights(dataset):
    similarity_scores = dataset["confidence"]  # 데이터셋에서 유사도 점수를 가져옴
    # 유사도 점수를 [0, 1]로 정규화 후 가중치로 변환 (1에 가까울수록 가중치가 큼)
    weights = np.array(similarity_scores) / np.max(similarity_scores)
    return weights


In [11]:
from sklearn.model_selection import train_test_split

def remove_na(example):
    return all(v is not None for v in example.values())

cleaned_dataset = dataset.filter(remove_na)

train_test_split = cleaned_dataset['train'].train_test_split(test_size=0.2)

train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

# train_dataset에 가중치 추가
weights = calculate_weights(train_dataset)
train_dataset = train_dataset.add_column("weights", weights)

tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_test_dataset = test_dataset.map(generate_and_tokenize_prompt)

Filter:   0%|          | 0/9873 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/7898 [00:00<?, ? examples/s]

Map:   0%|          | 0/7898 [00:00<?, ? examples/s]

Map:   0%|          | 0/1975 [00:00<?, ? examples/s]

In [12]:
idx = 8

eval_prompt = f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Given a original competitive programming problem, generate a concrete planning to solve the problem.
The provided retrieved question and code are examples to help generate the plan.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
### Original Problem to be solved :
{test_dataset["similarQuestion"][idx]}

### Retrieved Problem (similar question) :
{test_dataset["question"][idx]}

### Retrieved code snippet (related to the retrieved question) :
{test_dataset["snippet"][idx]}

### Similarity score (between original problem and retrieved question) :
{test_dataset["confidence"][idx]}
--------------------
Important: You should give only the planning to solve the problem. Do not add extra explanation or any code.
Let's think step by step.

### Planning to solve the problem in 6 lines or less:<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""


In [13]:
model_input = tokenizer(eval_prompt, return_tensors="pt").to(device)

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=125)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



system
Given a original competitive programming problem, generate a concrete planning to solve the problem.
The provided retrieved question and code are examples to help generate the plan.
user
### Original Problem to be solved :
How can I create a 2D image plot using the gnuplot module in python, where the data is represented as a matrix?

### Retrieved Problem (similar question) :
How do I do image plots in the gnuplot module for python?

### Retrieved code snippet (related to the retrieved question) :
#!/usr/bin/python

import Gnuplot as gp
import numpy as np

g = gp.Gnuplot(debug=1)

m = np.matrix('4 4 4 1; 3 2 1 1; 1 2 3 1; 0 0 0 0')
gdat = gp.GridData(m, inline=True, binary=False)
g('set style data image')
g.plot(gdat)


### Similarity score (between original problem and retrieved question) :
0.85720915
--------------------
Important: You should give only the planning to solve the problem. Do not add extra explanation or any code.
Let's think step by step.

### Planning to solve

In [14]:
# train_dataset[1]
# 가중치 컬럼이 있는지 확인
print(tokenized_train_dataset.column_names)  # "weights" 컬럼이 포함되어 있어야 합니다.


['question_id', 'question', 'snippet', 'similarQuestion', 'confidence', 'plan', 'weights', 'input_ids', 'attention_mask', 'labels']


In [15]:
from transformers import DataCollatorForLanguageModeling

class WeightedDataCollator(DataCollatorForLanguageModeling):
    def __call__(self, features):
        batch = super().__call__(features)
        # 가중치가 없는 경우 기본값 1.0을 설정
        batch["weights"] = torch.tensor([f.get("weights", 1.0) for f in features], dtype=torch.float)
        return batch


from transformers import Trainer
import torch

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # 입력 데이터에서 가중치 추출
        weights = inputs.pop("weights", None)  # weights를 inputs에서 제거하고 따로 저장

        # 모델의 forward 호출 (가중치 없이)
        outputs = model(**inputs)
        loss = outputs.loss

        # 가중치가 있는 경우 가중치를 적용하여 손실 계산
        if weights is not None:
            weighted_loss = loss * weights
            loss = weighted_loss.mean()  # 배치 차원의 평균 손실 계산

        return (loss, outputs) if return_outputs else loss

data_collator = WeightedDataCollator(tokenizer, mlm=False)


In [16]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [17]:
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 6678528 || all params: 755953664 || trainable%: 0.8834573225906078


/usr/local/lib/python3.10/dist-packages/peft/tuners/tuners_utils.py:500: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


In [18]:
project = "StaQC-plan"
base_model_name = "Llama-3.2-1B-Instruct"
run_name = f"{project}-{base_model_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"
output_dir = f"/content/drive/MyDrive/{run_name}"

tokenizer.pad_token = tokenizer.eos_token

# trainer = transformers.Trainer(
trainer = WeightedTrainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=5,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        max_steps=300,
        learning_rate=2e-5,
        logging_steps=50,
        fp16=True,
        optim="paged_adamw_8bit",
        # lr_scheduler_type="cosine",
        # loggig_dir="./logs",
        save_strategy="steps",
        save_steps=100,
        # save_total_limit=2,
        evaluation_strategy="steps",
        eval_steps=50,
        do_eval=True,
        load_best_model_at_end=True,
        report_to="none",
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}",
    ),
    data_collator=data_collator,
    # data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train()




/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  retu

Step,Training Loss,Validation Loss
50,2.192000,1.269962
100,1.100700,1.019246
150,1.011600,0.995518
200,0.995100,0.985022
250,0.979300,0.979306
300,0.974300,0.977297


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:227: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:227: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embeddi

TrainOutput(global_step=300, training_loss=1.2088367335001629, metrics={'train_runtime': 1678.6882, 'train_samples_per_second': 1.43, 'train_steps_per_second': 0.179, 'total_flos': 7224051105792000.0, 'train_loss': 1.2088367335001629, 'epoch': 0.30387439858191945})

In [20]:
model_input = tokenizer(eval_prompt, return_tensors="pt").to(device)

ft_model = PeftModel.from_pretrained(model, "/content/drive/MyDrive/bbb/checkpoint-300")

ft_model.eval()
with torch.no_grad():
    print(tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=125)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



system
Given a original competitive programming problem, generate a concrete planning to solve the problem.
The provided retrieved question and code are examples to help generate the plan.
user
### Original Problem to be solved :
How can I create a 2D image plot using the gnuplot module in python, where the data is represented as a matrix?

### Retrieved Problem (similar question) :
How do I do image plots in the gnuplot module for python?

### Retrieved code snippet (related to the retrieved question) :
#!/usr/bin/python

import Gnuplot as gp
import numpy as np

g = gp.Gnuplot(debug=1)

m = np.matrix('4 4 4 1; 3 2 1 1; 1 2 3 1; 0 0 0 0')
gdat = gp.GridData(m, inline=True, binary=False)
g('set style data image')
g.plot(gdat)


### Similarity score (between original problem and retrieved question) :
0.85720915
--------------------
Important: You should give only the planning to solve the problem. Do not add extra explanation or any code.
Let's think step by step.

### Planning to solve

In [21]:
ft_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): LlamaForCausalLM(
          (model): LlamaModel(
            (embed_tokens): Embedding(128256, 2048)
            (layers): ModuleList(
              (0-15): 16 x LlamaDecoderLayer(
                (self_attn): LlamaSdpaAttention(
                  (q_proj): lora.Linear4bit(
                    (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=2048, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=2048, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
         

In [ ]:
# 1. Import the required modules.
# 2. Define a function `click` that opens a file using `askopenfilename`.
# 3. Inside `click`, use `os.path.split` to split the file path into a directory path.
# 4. Print the directory path.
# 5. Create a Tkinter GUI application.
# 6. Call the `click` function inside the GUI application.


In [22]:
# 로컬 디렉토리에 모델 저장

# 모델을 Hugging Face Hub에 저장
ft_model.push_to_hub("donghuna/llama-plan-1B")
tokenizer.push_to_hub("donghuna/llama-plan-1B")

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:227: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


adapter_model.safetensors:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/donghuna/llama-plan-1B/commit/d37b270284fc08736f05290bf215dd2069215d20', commit_message='Upload tokenizer', commit_description='', oid='d37b270284fc08736f05290bf215dd2069215d20', pr_url=None, pr_revision=None, pr_num=None)

In [23]:
tokenized_train_dataset

Dataset({
    features: ['question_id', 'question', 'snippet', 'similarQuestion', 'confidence', 'plan', 'weights', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 7898
})